In [1]:
import pyedflib
import numpy as np
import pandas as pd
import mne
from matplotlib.pyplot import figure
%matplotlib inline
import matplotlib.pyplot as plt

from mne.datasets import eegbci
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score
import pytorch_lightning as pl
from autoreject import AutoReject
from scipy.fft import fft, fftfreq, rfft, rfftfreq, irfft
import pickle
from sklearn.decomposition import PCA
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.ensemble import RandomForestClassifier

%pylab inline

Populating the interactive namespace from numpy and matplotlib


1. чтение файлов
2. разбиение на трейн и тест cross_val_score
3. уменьшение размерности
4. классификация

##  Чтение файлов

https://mne.tools/dev/generated/mne.datasets.eegbci.load_data.html

https://github.com/SpenderJ/Total-perspective-vortex/blob/main/pipeline.py

In [139]:
path = '/Users/ila/Desktop/Python/my projects/total-perspective-vortex/mne_data'
runs = [5, 9, 13] # Моторное исполнение: руки vs ноги
raw_names = []
for sub in range(30,31):
    if sub not in [88, 92, 100]: #битые файлы
        name = eegbci.load_data(sub, runs, path = path)
        raw_names.append(name)

In [140]:
# Подгружаем все файлы
all_data = []
for names in raw_names:
    for file in names:
        all_data.append(mne.io.read_raw_edf(file, preload=True, stim_channel='auto'))

# all_data.append(mne.io.read_raw_edf(raw_names[0][0], preload=True, stim_channel='auto'))

Extracting EDF parameters from /Users/ila/Desktop/Python/my projects/total-perspective-vortex/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S030/S030R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19839  =      0.000 ...   123.994 secs...
Extracting EDF parameters from /Users/ila/Desktop/Python/my projects/total-perspective-vortex/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S030/S030R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19839  =      0.000 ...   123.994 secs...
Extracting EDF parameters from /Users/ila/Desktop/Python/my projects/total-perspective-vortex/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S030/S030R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19839  =      0.000 ...   123.994 secs...


In [141]:
# Объединяем в один стэк данных для простоты работы
# у нас остались 64 датчика, только теперь они включают в себя объединенную инфу по каждому датчику по всем файлам
raw = mne.concatenate_raws(all_data)
# Стандартизируйте позиции и названия каналов
eegbci.standardize(raw)
# убрали точки в названии каналов и насторили порядок

In [142]:
raw.get_data().shape

(64, 59520)

In [143]:
raw.get_data().shape[1] / 90

661.3333333333334

In [144]:
19680 * 3 + 19840 * 3

118560

Montages содержат положения датчиков в 3D (x, y, z в метрах), которые могут быть назначены существующим данным ЭЭГ / МЭГ. 
Указывая расположение датчиков относительно мозга, Montages играют важную роль в вычислении прямого решения и обратных оценок.

Тут мы стантартизируем Montages в соответвие с обще принятым стандартом standard_1005

In [145]:
montage = mne.channels.make_standard_montage('standard_1005')
raw.set_montage(montage)

# raw.filter(7., 30., method='iir') # применяем фильтрацию из прошлого раздела

<RawEDF | S030R05.edf, 64 x 59520 (372.0 s), ~29.2 MB, data loaded>

events - это информация вида [продолждительность (диапазон частот дискритизации)| если указанно, то более детальная инфа про каналы |код метки 'T0', 'T1', 'T2' ]

to=1 (отдых)

t2=2 (ноги)

t1=3 (кулаки)


In [146]:
events, _ = mne.events_from_annotations(raw)
events[:3], events.shape

Used Annotations descriptions: ['T0', 'T1', 'T2']


(array([[   0,    0,    1],
        [ 672,    0,    2],
        [1328,    0,    1]]),
 (90, 3))

In [147]:
events[-1][0], 118560

(58720, 118560)

120 / 30 -время на один эксперемент? 120 сек и 30 заданй

У нас есть испытуемые, они прошли по 14 эксперементов, мы рассматриваем сейчас только 3 из 14. В каждом из этих эксперементов 30 заданий 
одинаковой длинны на сжатия рук/ног и отдых, они даются в хаотичном порядке. В каждом из 3 файлов инфа с 64 датчиков. Для простоты мы объединим их в один файл (raw), где будут 64 датчика с последовательно склеиных между собой частот из трех файлов. Дальше разбиваем на events. В нем кажому определенному дипазаону частоты дискретизации соответвует своя команда(рука/нога/отдых. а из этого получаем Эпохи, те подтягиваем реальные данные к меткам. Так как заданий везде по 30 заданий, то времени дается везде одинаково, след-во дипоазон в events везде одинаковый? tmin, tmax для того что бы задать более точный диапазон на эксперемент.

In [148]:
"""примерная продолжительность одного эксперемента. объединенная инфа по каждому датчику по всем файлам / на кол-во событий 
Продолжительность каждого из эксперементов примерно одинакова.
""" 
raw.get_data().shape[1] / events.shape[0], events[3][0]-events[2][0]

(661.3333333333334, 656)

In [149]:
# pick_types отбирает нам хорошие и плохие каналы 
# exclude='bads' - добавлять плохие каналы в raw.info['bads']
picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

In [150]:
picks, raw.info['bads']

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]),
 [])

### epochs разделяет данные дескритизации в соответвии с метками классов

In [151]:
event_ids = dict(rest=1, hands=2, feet=3)
event_ids

{'rest': 1, 'hands': 2, 'feet': 3}

In [152]:
tmin, tmax = 0.1, 4.
epochs = mne.Epochs(raw, events, event_ids,tmin, tmax, picks=picks, preload=True, proj=True, baseline=None)

Not setting metadata
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Loading data for 90 events and 625 original time points ...
0 bad epochs dropped


In [153]:
epochs

Number of events,90
Events,feet: 23hands: 22rest: 45
Time range,0.100 – 4.000 sec
Baseline,off


In [155]:
ar = AutoReject() # для автоматического отклонения неудачных испытаний и восстановления неисправных датчиков
epochs_clean = ar.fit_transform(epochs) 

In [ ]:
epochs = epochs_clean

In [124]:
labels = epochs.events[:, -1] # возращаемся к нашим events и берем от туда целевую переменную 
labels[:5]

array([1, 3, 1, 2, 1])

данные epochs_data_train имет вид:

эксперемент(ступня/калак/отдых) | 64 тдатчика со всей инфой  | частота дискретизации

In [125]:
epochs_data_train = epochs.get_data() # 
epochs_data_train.shape, len(labels)

((2392, 64, 625), 2392)

In [ ]:
# 220 см

In [ ]:
# 1 и 7 метра 

In [102]:
# 118560, events[-1][0], 625 * 180

In [91]:
# epochs_data_train[0][0]

## Фурье преобразование 

In [126]:
SAMPLE_RATE = 160 #частота дискретизации 
DURATION = tmax - tmin # продолжитьельность в секундах
N = int(SAMPLE_RATE * DURATION)
mas1 = []
for exper in epochs_data_train:
    mas2 = np.array([])
    for datchik in exper:
        maxs = datchik.max()
        norm_raw = datchik /  maxs
        if not np.isfinite(norm_raw).all():
            mas2 = np.vstack((mas2, np.array([0] * len(mas2[0]))))
            continue
        yf = rfft(norm_raw)
        xf = rfftfreq(N, 1 / SAMPLE_RATE) 
        yf2 = [0 if np.abs(i) < 10 else i for i in yf]
        new_sig = irfft(yf2)
        if len(mas2) == 0:
            mas2 = new_sig * maxs
        else:
            mas2 = np.vstack((mas2, new_sig * maxs))
    mas1.append(mas2)
mas1 = np.array(mas1)

<ipython-input-126-2562ab359371>:9: RuntimeWarning: divide by zero encountered in true_divide
  norm_raw = datchik /  maxs
<ipython-input-126-2562ab359371>:9: RuntimeWarning: invalid value encountered in true_divide
  norm_raw = datchik /  maxs


IndexError: index 0 is out of bounds for axis 0 with size 0

In [24]:
epochs_data_train = mas1

In [ ]:
mas1.shape

## Построение Pipeline

1. разбраться с cross_val_score
2.  разбраться с CSP уменьшает размерность? 
3.  разбраться с LDA предсказательная модель 

LDA - Линейный дискриминантный анализ - алгоритм машинного обучения классификации.
https://machinelearningmastery.com/linear-discriminant-analysis-with-python/

# Должны быть разнве pipeline, для обучения и для предсказания  
Работа на pipeline c предсказанием должна быть не больше 2 секнд

Результат обучения на всей выборке должен быть от >= 75 %, но что бы получть максимум былов нужно больше 90%

In [26]:
SEED = 42
pl.seed_everything(SEED)
random.seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

Global seed set to 42


In [67]:
cv = ShuffleSplit(1, test_size=0.3, random_state=42)
lda = LDA()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

In [39]:
# clf = Pipeline([('CSP', csp), ('LDA', lda)])
# clf.fit(epochs_data_train, labels)

In [38]:
clf.predict(epochs_data_train)

array([1, 1, 1, ..., 1, 1, 1])

In [36]:
clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores_lda = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=-1)
mean_scores_lda, std_scores_lda = np.mean(scores_lda), np.std(scores_lda)
mean_scores_lda, std_scores_lda

(0.5092592592592593, 0.011328987866673171)

In [ ]:
rfc = RandomForestClassifier()
crf = Pipeline([('CSP', csp), ('RFC', rfc)])
scores_crf = cross_val_score(crf, epochs_data_train, labels, cv=cv, n_jobs=-1)
mean_scores_crf, std_scores_crf = np.mean(scores_crf), np.std(scores_crf)
mean_scores_crf, std_scores_crf

In [59]:
class Reshape:
    def fit(self, data, lab = None):
        pass
        
    def transform(self, data = None):
        lens = len(data)
        return data.reshape(lens, -1)
    
    def fit_transform(self, data, lab = None):
        self.fit(data)
        return self.transform(data)
    

In [60]:
from pca import My_PCA


In [156]:
data_hand_feet = epochs_data_train[np.logical_or(labels == 3, labels == 2)]
data_hand_feet.shape

(1210, 64, 625)

In [157]:
label_hand_feet =[1 if i == 2 else 0 for i in labels[np.logical_or(labels == 3, labels == 2)]]
len(label_hand_feet)

1210

In [ ]:
# label_hand_feet

In [ ]:
# epochs_data_train

максимальные значения были под 0.55 - это без каких либо фильтраций)))

In [164]:
%%time

pca = UnsupervisedSpatialFilter(PCA(15))
resh = Reshape()
pca2 = PCA(n_components=15, random_state=42)
pld = Pipeline([('PCA', pca), ('Reshape', resh),('PCA2', pca2), ('LDA', lda)])
scores_pld = cross_val_score(pld, data_hand_feet, label_hand_feet, cv=cv, n_jobs=-1)
mean_scores_pld, std_scores_pld = np.mean(scores_pld), np.std(scores_pld)
mean_scores_pld, std_scores_pld

CPU times: user 193 ms, sys: 815 ms, total: 1.01 s
Wall time: 8.83 s


(0.5977961432506887, 0.0)

In [162]:
with open('my_model.pickle', 'wb') as f:
    pickle.dump(pld, f)

In [66]:
data_hand_feet.shape, len(label_hand_feet)

((45, 64, 625), 45)

1. Разобраться с PCA
2. Доделать бонусы
3. Сдать проект

In [ ]:
# !pip install pytorch_lightning

In [ ]:
%%time
pl.seed_everything(SEED)
random.seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pca = UnsupervisedSpatialFilter(PCA(10))
resh = Reshape()
my_pca = My_PCA(n_components=35)
pld = Pipeline([('PCA', pca), ('Reshape', resh),('My_PCA', my_pca), ('LDA', lda)])
scores_pld = cross_val_score(pld, epochs_data_train, labels, cv=cv, n_jobs=-1)
mean_scores_pld, std_scores_pld = np.mean(scores_pld), np.std(scores_pld)
mean_scores_pld, std_scores_pld

In [ ]:
epochs_data_train.shape

In [ ]:
# filter(lambda v: v==v, x)

In [ ]:
pca = UnsupervisedSpatialFilter(PCA(10))
pca_data = pca.fit_transform(data_hand_feet)
pca_data.shape

In [ ]:
resh = Reshape()
pca_data_resh = resh.fit_transform(pca_data)
pca_data_resh.shape

In [ ]:
pca2 = PCA(n_components=35)
pca_data_resh_pc = pca2.fit_transform(pca_data_resh)
pca_data_resh_pc.shape

In [ ]:
label_hand_feet

In [ ]:
signs = [1 if i >= 0 else -1 for i in pca_data_resh_pc[0]]
len(signs)

In [ ]:
pca_data_resh_pc

In [ ]:
my_pca = My_PCA(n_components=35)
pca_data_resh_my = my_pca.fit_transform(pca_data_resh)
pca_data_resh_my.shape

In [ ]:
1 + 1

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
pca_data_resh_pc

In [ ]:
for i in range(len(pca_data_resh_my)):
    for j in range(len(pca_data_resh_my[i])):
        if pca_data_resh_pc[i][j] >= 0:
            pca_data_resh_my[i][j] = abs(pca_data_resh_my[i][j])
        else:
            pca_data_resh_my[i][j] = -abs(pca_data_resh_my[i][j])
pca_data_resh_my               

In [ ]:
lda.fit(pca_data_resh_my[:2200], labels[:2200])
accuracy_score(lda.predict(pca_data_resh_my[2200:]), labels[2200:])

In [ ]:
pca_data_resh_pc.shape

In [ ]:
lda.fit(pca_data_resh_pc[:35], label_hand_feet[:35])
accuracy_score(lda.predict(pca_data_resh_pc[35:]), label_hand_feet[35:])

# Бонусы 
Их всего 3 пункта, так что надо выполнить каждый из них
1. Провести обучение и все эти процедуры на другом эксперементе
2. Хороший препроцесинг является плюсом, преобразование Фурье и Вейвлета

https://proglib.io/p/preobrazovaniya-fure-dlya-obrabotki-signalov-s-pomoshchyu-python-2020-11-03

3. Вроде как тут все ок, ты реализовал собственную коварационную матрицу 

## Визуализация

In [ ]:
from sklearn import manifold

### CSP

In [ ]:
data_2d_csp = csp.fit_transform(epochs_data_train, labels)
fig, ax = plt.subplots(figsize=(10,6))
classes = ['rest', 'feet', 'hands']
scatter = ax.scatter(data_2d_csp[:, 0], data_2d_csp[:, 1], c=labels)
plt.legend(handles=scatter.legend_elements()[0], labels=classes)

### TSNE

In [ ]:
nsamples, nx, ny = epochs_data_train.shape
d2_epochs_data_train = epochs_data_train.reshape((nsamples,nx*ny))

In [ ]:
tsne = manifold.TSNE(n_components = 2, random_state = 0)
data_2d_tsne = tsne.fit_transform(d2_epochs_data_train, labels)

In [ ]:
pylab.figure(figsize = (10, 6))
pylab.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], c = labels)